In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from new_experiments import run_simulation
from concurrent.futures import ProcessPoolExecutor
from IPython.display import display, clear_output

sys.path.append(os.getcwd())


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
def run_experiment():
    outputs = []
    ret_df = None
    for dataset in [3]:
        if not os.path.exists(f"./zero_imputation_experiments/DS{dataset}/"):
            os.makedirs(f"./zero_imputation_experiments/DS{dataset}/")
        # Run for first iteration to prevent race condition
        res = run_simulation(
            dataset=dataset,
            sergio=True,
            saucie=True, 
            scScope=True, 
            deepImpute=True, 
            magic=True, 
            genie=True,
            arboreto=False,
            pearson=False,
            roc=True,
            precision_recall_k=False,
            run_with_regs=False,
            iteration=0
        )
        clear_output()
        if ret_df is None:
            ret_df = pd.DataFrame(columns=res.keys())
        new_df = pd.DataFrame([res], columns=res.keys())
        ret_df = pd.concat([ret_df, new_df], ignore_index=True)
        #write to temp file
        ret_df.to_csv("zero_imputation_experiments/imputation_results.csv", index=False)
        with ProcessPoolExecutor(max_workers=3) as executor:
            futures = []
            for i in range(1, 30):
                futures.append(executor.submit(run_simulation, 
                        dataset=dataset,
                        sergio=(i == 0),
                        saucie=True, 
                        scScope=True, 
                        deepImpute=True, 
                        magic=True, 
                        genie=True,
                        arboreto=False,
                        pearson=False,
                        roc=True,
                        precision_recall_k=False,
                        run_with_regs=False,
                        iteration=i
                    ))
                clear_output()
            for future in tqdm(futures):
                res = future.result()
                clear_output(wait=True)
                if ret_df is None:
                    ret_df = pd.DataFrame(columns=res.keys())
                new_df = pd.DataFrame([res], columns=res.keys())
                ret_df = pd.concat([ret_df, new_df], ignore_index=True)
                #write to temp file
                ret_df.to_csv("zero_imputation_experiments/imputation_results.csv", index=False)
    return

In [ ]:
run_experiment()

### Other Imputation Methods Experimentation

In [8]:
import os
for dataset in [1,2,3]:
    if not os.path.exists(f"./zero_imputation_experiments/DS{dataset}/"):
        os.makedirs(f"./zero_imputation_experiments/DS{dataset}/")
    if not os.path.exists(f"./zero_imputation_experiments/DS{dataset}/DS6_noisy.npy"):
        res = run_simulation(
            dataset=dataset,
            sergio=True,
            saucie=False, 
            scScope=False, 
            deepImpute=False, 
            magic=False, 
            genie=False,
            arboreto=False,
            pearson=False,
            roc=False,
            precision_recall_k=False,
            run_with_regs=False,
            iteration=0
        )
        clear_output()

scVI

In [8]:
from experiment_utils import run_scvi
import numpy as np
import pandas as pd

def fetch_target_regs(dataset):
    if dataset == 1:   
        target_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Interaction_cID_4.txt'
        regs_path = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Regs_cID_4.txt'
    elif dataset == 2:
        target_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Interaction_cID_5.txt'
        regs_path = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Regs_cID_5.txt'
    else:
        target_file = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'
        regs_path = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt'
    return target_file, regs_path    

def scvi_impute():
    ret_df = None
    for dataset in [1, 2, 3]:
        save_path = f"./zero_imputation_experiments/DS{dataset}/"
        y = np.load(save_path + "/DS6_noisy.npy")
        target_file, regs_path = fetch_target_regs(dataset)
        with ProcessPoolExecutor(max_workers=4) as executor:
            futures = []
            for i in range(8):
                futures.append(executor.submit(run_scvi, 
                    data=y, 
                    save_path=save_path, 
                    it=i, 
                    file_extension=f"_iter{i}",
                    target_file=target_file
                ))
                clear_output()
            for future in tqdm(futures):
                vim, it = future.result()
                res = {
                    "dataset": dataset,
                    "method": "scvi",
                    "roc": vim,
                    "iteration": it }
                if ret_df is None:
                    ret_df = pd.DataFrame(columns=res.keys())
                new_df = pd.DataFrame([res], columns=res.keys())
                ret_df = pd.concat([ret_df, new_df], ignore_index=True)
                ret_df.to_csv("zero_imputation_experiments/scvi_imputation_results.csv", index=False)  

In [ ]:
scvi_impute()

knn-smoothing

In [24]:
from experiment_utils import run_knn

def run_smoothing():
    ret_df = None
    for dataset in [1,2,3]:
        save_path = f"./zero_imputation_experiments/DS{dataset}/"
        y = np.load(save_path + "/DS6_noisy.npy")
        target_file, regs_path = fetch_target_regs(dataset)
        with ProcessPoolExecutor(max_workers=4) as executor:
            futures = []
            for i in range(8):
                futures.append(executor.submit(run_knn, 
                    data=y,
                    k=32,
                    save_path=save_path, 
                    it=i, 
                    file_extension=f"_iter{i}",
                    target_file=target_file
                ))
                clear_output()
            for future in tqdm(futures):
                vim, it = future.result()
                res = {
                    "dataset": dataset,
                    "method": "knn",
                    "roc": vim,
                    "iteration": it }
                if ret_df is None:
                    ret_df = pd.DataFrame(columns=res.keys())
                new_df = pd.DataFrame([res], columns=res.keys())
                ret_df = pd.concat([ret_df, new_df], ignore_index=True)
                ret_df.to_csv("zero_imputation_experiments/knn_imputation_results.csv", index=False)  

In [25]:
run_smoothing()

  0%|                                                           | 0/8 [00:00<?, ?it/s]/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.9/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.9/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.9/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/Yos

Performing kNN-smoothing v2.1 with k=32, d=10, and dither=0.030...
Performing kNN-smoothing v2.1 with k=32, d=10, and dither=0.030...
Performing kNN-smoothing v2.1 with k=32, d=10, and dither=0.030...
Performing kNN-smoothing v2.1 with k=32, d=10, and dither=0.030...
Step 1/5: Smooth using k=2
Step 1/5: Smooth using k=2
Step 1/5: Smooth using k=2
Step 1/5: Smooth using k=2
	PCA took 25.5 s.
	The fraction of variance explained by the top 10 PCs is 52.8 %.
	Calculating pair-wise distance matrix took 0.2 s.
	PCA took 26.5 s.
	The fraction of variance explained by the top 10 PCs is 52.8 %.
	Calculating pair-wise distance matrix took 0.1 s.
	PCA took 27.2 s.
	PCA took 27.5 s.
	The fraction of variance explained by the top 10 PCs is 52.8 %.
	Calculating pair-wise distance matrix took 0.3 s.
	The fraction of variance explained by the top 10 PCs is 52.8 %.
	Calculating pair-wise distance matrix took 0.1 s.
	Calculating the smoothed expression matrix took 1.9 s.
Step 2/5: Smooth using k=4
	Calc

100%|██████████| 1133/1133 [00:00<00:00, 441034.47it/s]


Elapsed time: 933.20 seconds
Elapsed time: 932.14 seconds


100%|██████████| 1133/1133 [00:00<00:00, 578401.46it/s]


...

100%|██████████| 1133/1133 [00:00<00:00, 600397.53it/s]


Elapsed time: 905.83 seconds


100%|██████████| 1133/1133 [00:00<00:00, 591283.62it/s]


Elapsed time: 900.36 seconds


100%|██████████████████████████████████████████████████| 8/8 [35:38<00:00, 267.37s/it]
